In [1]:
import os
import glob 
import mteb
import json
import yaml
import shutil
import pandas as pd
from pathlib import Path
from sentence_transformers import SentenceTransformer

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DirectoryLoader


In [3]:
# ai_config/load_ai_config.yaml 파일 로드
with open('../ai_config/load_embedding_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# 모델 디렉토리 존재 여부 확인 및 생성 
results = []
for model_config in config['models']:
    model_path = f"../{model_config['target_dir']}"
    
    # 모델 디렉토리가 존재하는지 확인
    if not os.path.exists(model_path):
        print(f"경고: {model_path} 경로가 존재하지 않습니다. 해당 모델을 건너뜁니다.")
        continue
        
    print(f"모델 평가 중: {model_path}")
    
    try:
        revision = "main"
        model = mteb.get_model(model_path, revision=revision, trust_remote_code=True)
        
        tasks = mteb.get_tasks(tasks=["AutoRAGRetrieval", "KorSTS"])
        evaluation = mteb.MTEB(tasks=tasks)
        
        # 각 모델별 결과 저장 폴더 생성
        model_name = model_path.split('/')[-1]
        os.makedirs(f"results/{model_name}", exist_ok=True)
        
        result = evaluation.run(model, output_folder=f"results/{model_name}")
        results.append(result)
        
    except Exception as e:
        print(f"오류 발생: {model_path} 모델 평가 중 실패") 
        print(f"오류 메시지: {str(e)}")
        continue


Loader not specified for model ../ai_model_dir/hf/kanana/kanana-nano-2.1b-embedding, loading using sentence transformers.
No sentence-transformers model found with name ../ai_model_dir/hf/kanana/kanana-nano-2.1b-embedding. Creating a new one with mean pooling.


모델 평가 중: ../ai_model_dir/hf/kanana/kanana-nano-2.1b-embedding


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- AutoRAGRetrieval, s2p

STS

- KorSTS, s2s

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Error while evaluating AutoRAGRetrieval: Kanana2VecModel.forward() missing 1 required positional argument: 'pool_mask'


오류 발생: ../ai_model_dir/hf/kanana/kanana-nano-2.1b-embedding 모델 평가 중 실패
오류 메시지: Kanana2VecModel.forward() missing 1 required positional argument: 'pool_mask'


In [ ]:
# Define the sentence-transformers model name
revision = "main"  # 또는 원하는 커밋 해시, 태그 등

model = mteb.get_model(EMBEDDING_MODEL_PATH, revision=revision)

tasks = mteb.get_tasks(tasks=["AutoRAGRetrieval", "KorSTS"])
evaluation = mteb.MTEB(tasks=tasks)
results = evaluation.run(model, output_folder=f"results/BGE-m3-ko")

# 결과 출력
for result in results:
    print(f"Task: {result.task_name}")
    for metric, score in result.scores.items():
        print(f"  {metric}: {score}")

In [56]:
import shutil
import glob, os
from pathlib import Path
if not os.path.exists("./results/flat"):
    os.makedirs("./results/flat", exist_ok=True)

if not os.path.exists("./results/flat/BGE-m3-ko"):
    os.makedirs("./results/flat/BGE-m3-ko", exist_ok=True)


for file in Path("./results/BGE-m3-ko").rglob("*.json"):
    if file.name.endswith("KorSTS.json"):
        shutil.copy(file, "./results/flat/BGE-m3-ko/KorSTS.json")
    elif file.name.endswith("AutoRAGRetrieval.json"):
        shutil.copy(file, "./results/flat/BGE-m3-ko/AutoRAGRetrieval.json")

In [ ]:
for file in Path("./results/BGE-m3-ko").rglob("*.json"):
    print(file)


In [ ]:
import json
import pandas as pd
from pathlib import Path

# 결과 폴더 경로
score_dir = Path("./results/test_score")

# 파일별로 추출할 지표 정의
score_keys = {
    "KorSTS.json": ["cosine_spearman"],
    "AutoRAGRetrieval.json": ["recall_at_1", "recall_at_5", "mrr_at_10"]
}

# 결과 저장 리스트
rows = []

# 모든 json 파일 순회
for json_file in score_dir.glob("*.json"):
    if json_file.name == "model_meta.json":  # ← .name 속성으로 비교!
        continue
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
        scores = data.get("scores", {})
        if isinstance(scores, dict) and "test" in scores and isinstance(scores["test"], list):
            scores = scores["test"][0]
        metrics = score_keys.get(json_file.name, [])
        filtered = {metric: scores.get(metric, None) for metric in metrics}
        filtered["task"] = json_file.stem
        rows.append(filtered)

# DataFrame 생성
df = pd.DataFrame(rows)
cols = ["task", "cosine_spearman", "recall_at_1", "recall_at_5", "mrr_at_10"]
df = df.reindex(columns=cols)

print(df)

In [ ]:
df.fillna("없음", inplace=True)   
df['model'] = "multilingual-e5-large-instruct"

In [103]:

df.to_csv("./results/csv/multilingual-e5-large-instruct.csv", index=False)

In [96]:
csv_path = Path("./results/csv").rglob("*.csv")

In [ ]:
import pandas as pd
from pathlib import Path

# 모든 csv 파일 경로 찾기
csv_files = list(Path("./results/csv").glob("*.csv"))

# DataFrame 리스트에 모두 읽기
df_list = [pd.read_csv(f) for f in csv_files]

# 하나의 DataFrame으로 합치기
combined_df = pd.concat(df_list, ignore_index=True)

# 합친 결과를 새 파일로 저장
combined_df.to_csv("./results/csv/combined_results.csv", index=False)

print("모든 csv 파일을 합쳐서 combined_results.csv로 저장했습니다.")

In [ ]:
combined_df

In [ ]:
parquet_docs = loader.load()

In [44]:
csv_docs = loader.load()

# 임베딩 모델 생성
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_PATH)

# 문서 텍스트 추출 (Document 객체라면 .page_content 사용)
texts = [doc.page_content for doc in csv_docs]

# FAISS 인덱스 생성
vectorstore = FAISS.from_texts(texts, embedding_model)

# 인덱스 저장
vectorstore.save_local("../data/vectorstore/faiss_index")


In [6]:
embedding = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_PATH,
    model_kwargs={"device": "cpu"},                    # GPU 사용 안함
    encode_kwargs={"normalize_embeddings": True}       # 임베딩 정규화 (권장)
)
vectorstore = FAISS.load_local(VECTORSTORE_PATH, embedding, allow_dangerous_deserialization=True)

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})  # top-3 문서 검색

In [ ]:
retriever.get_relevant_documents("개인정보 유출이란?")  # 검색 쿼리

In [ ]:
# 검색 쿼리 입력
query = "개인정보"

# top-k 유사 문서 검색
docs = vectorstore.similarity_search(query, k=3)

# 결과 출력
for i, doc in enumerate(docs):
    print(f"결과 {i+1}:\n{doc.page_content}\n")
